# PROJET IA

## Partie 1

### Importation des librairies nécessaires :

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from sklearn.model_selection import train_test_split
from osgeo import gdal
from IPython.display import HTML
from base64 import b64encode
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from matplotlib import colors

Pour le traitement des données géospatiales nous avons besoins d'une librairie spéciale nommée GDAL (geos python lybrary). Cette dernière nécessite une installation particulière.
Les commandes ci-dessous sont celles requises pour l'installation de GDAL sur un environnement MAC OS X. Elle correspond à une installattion de GDAL via l'utilitaire de packages brew pour une version de python 3.11 et la version correspondante de GDAL 3.6.4_1.

In [ ]:
# !brew install gdal
# !pip download GDAL
# !tar -xpzf GDAL-3.6.4.tar.gz
# !cd GDAL-<version of GDAL>
# !python setup.py build_ext --gdal-config /usr/local/Cellar/gdal/3.6.4_1/bin/gdal-config
# !python setup.py build
# !python setup.py install

### Préparaation pour l'importation du dataset

In [ ]:
# nom de la time series des données géospatiales (format CSV)
dataset = "time_series.csv"

# mapping des numéros de classes avec les labels et des couleurs (pour le plotting)
classe_info = {
  'not identified':           {'value':0, 'color': '#000000'},
  'soybean':                  {'value':1, 'color': '#ffe32e'},
  'maize':                    {'value':2, 'color': '#FF0000'},
  'uncultivated soil':        {'value':3, 'color': '#a9b0b0'},
  'coffee':                   {'value':4, 'color': '#75781f'},
  'beans':                    {'value':5, 'color': '#e5eb34'},
  'wheat':                    {'value':6, 'color': '#ff24e5'},
  'sorghum':                  {'value':7, 'color': '#a80a96'},
  'millet':                   {'value':8, 'color': '#fa73eb'},
  'eucalyptus':               {'value':9, 'color': '#c75e0e'},
  'pasture':                  {'value':10, 'color': '#fff68f'},
  'hay':                      {'value':11, 'color': '#c9cf91'},
  'grass':                    {'value':12, 'color': '#12e362'},
  'crotalari':                {'value':13, 'color': '#12e362'},
  'maize+crotalari':          {'value':14, 'color': '#f77159'},
  'cerrado':                  {'value':15, 'color': '#5e2e10'},
  'conversion area':          {'value':16, 'color': '#12e0e3'},
  'cotton':                   {'value':17, 'color': '#0000FF'},
  'ncc':                      {'value':18, 'color': '#12e362'},
  'brachiaria':               {'value':19, 'color': '#12e362'},
}

# récupération des labels
classes = {x : y.get('value') for x, y in classe_info.items()}

# récupération des couleurs
classe_colors = [y.get('color') for x, y in classe_info.items()]

# récupération des feaures (EVI)
features = ['red', 'nir', 'swir']
# récupération du nombre de features
n_features = len(features)

# définir la taille d'une séquence
sequence_size = 30

# définition d'un chemin pour la création du dossier de logs pour l'entrainement du modèle
model_dir = './logs'

### Importation du dataset

In [ ]:
df = pd.read_csv(dataset)
df.head()

In [ ]:
# mapping des numéros de classe avec les labels prédéfinies
df['class_name'] = df.apply(lambda row: list(classes.keys())[list(classes.values()).index(row['class'])], axis = 1)
# convertir la colonne date en format datetime compréhenssible pour le modèle
df['date'] = pd.to_datetime(df['date'])
df.head()

### Analyses

In [ ]:
# récupéraation des sous-times series du dataframe
points = df.id.unique()

# pour les 7 premières times series récupérées
for point in points[:7]:
    point_df = df[df['id'] == point]                            # récupérer l'id qui servira de coordonée ordinale
    point_df = point_df.sort_values(by=['date'])                # trier par date (de la plus ancienne à la plus récente)
    ax = point_df.plot(x='date', y=features, figsize=(20, 5))   # mettre sur l'axe x les dates, l'aaxe y les valeurs de l'IEV (features)

    # ploter l'axe
    axes1 = plt.gca()
    axes2 = axes1.twiny()   
    
    class_names = point_df['class_name'].tolist()                               # récupérer les labels correspondants à chaque date
    axes2.set_xticks(np.arange(len(class_names)))                               # mettre les labels correspondants au niveu du titre
    axes2.set_xticklabels(class_names, rotation=50, fontsize=12, minor=False)   # modifier l'affichaage de l'axe des x

    # définir les titres des axes
    axes1.set_ylabel("Features")
    axes1.set_xlabel("Image Date")
    axes2.set_xlabel("Land Use/Land Cover")

## Partie 2

### Preprocessing

- Utilisation de tensorflow poour le preprocessing des données
- Passer les feaatures au format numérique (float) -> X
- Passer les labels au format numérique -> y

In [ ]:
X = []
y = []

# pour chaque time series :
for point in points:
    point_df = df[df['id'] == point]                # récuéprer la partie du DF correspondant à la time series
    point_df = point_df.sort_values(by=['date'])    # trier de la date la plus ancienne à la plus récente
    
    x_values = point_df[features].to_numpy()        # mettre les features dans un array numpy
    y_values = point_df['class'].tolist()           # mettre les labels dans une liste


    # utiliser tensorflow pour le preproecssing des array en les passant en array 2D pour le modèle.
    x_values = tf.keras.preprocessing.sequence.pad_sequences([x_values], 
                                                             maxlen=sequence_size, dtype='float32')[0]
    y_values = tf.keras.preprocessing.sequence.pad_sequences([y_values], 
                                                             maxlen=sequence_size, 
                                                             value=classes.get('not identified'), dtype='float32')[0]
    
    X.append(x_values)  # ajouter les features preprocessées à X
    
    # créer un array rempli de 0 et le reemplacer par les valeurs des labels
    labels = []
    for y_value in y_values:
        values = np.zeros(len(classes))
        np.put(values, [y_value], [1])
        labels.append(values)
        
    y.append(labels)    # ajouter les labels préprocéssées à y

# convertir lese deux listes en array (nous avons donc dess array contenant des 2D array de feeatures et de labels)
X = np.array(X)
y = np.array(y)

# redimensionner l'array des X
X = X.reshape((X.shape[0], X.shape[1], n_features))

X.shape, y.shape

### Split du dataset

In [ ]:
# séparation de dataset en jeu d'entrainement, de validation et de test
# utilisation de 80% de la time series pour l'entrainement et de 10% pour la validation et 10% pour le test

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)
X_validation, X_test, y_validation, y_test = train_test_split(X_validation, y_validation, test_size=0.5)

print("Train: ", len(X_train), "\nValidation: ", len(X_validation), "\nTest:", len(X_test))

In [ ]:
X_train.shape, y_train.shape

## Partie 3

### Création du modèle

Le modèle que nous avons décidé d'utiliser est un modèle de deep leaarning de type LSTM :

- Parmis les modèles existants dans l'état de l'art, les modèles de type LSTM pour la prédiction de Time Series sembleent être les plus performants.
- Nous n'avons pas souhaité utiliser un framework d'entrainement (de type API, comme celle de PyTorch ou TF) car nous avons pensé avoir les connaissances suffisantes pour définir nous même l'aarchitecture de notre modèle.
- Ainsi nous avons pu meettre en oeuvre une structure de modèle de deep learning en restant simple, cee qui nous permet de contrôler nos outputs, de faire des modifications de structures si nécessaires et d'avoir une idée des outputs à chaque fin de layer.

In [ ]:
def LSTM(n_classes, sequence_size, n_features):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.LSTM(200, input_shape=(sequence_size, n_features)))
    
    model.add(tf.keras.layers.RepeatVector(sequence_size))
    
    model.add(tf.keras.layers.LSTM(200, activation='relu', return_sequences=True))
    
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(100, activation='relu')))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_classes)))
    
    model.add(tf.keras.layers.Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
              
model = LSTM(n_classes=len(classes), sequence_size=sequence_size, n_features=n_features)
              
model.summary()

tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png') # enregistrement de la structure au format png

### Paramétrage d'entrainement

In [ ]:
checkpoint_path = "{dir}/model.ckpt".format(dir=model_dir) # chemin d'enregistrement des checkpointt pour reprendre l'entrainement en cas d'arret

latest = tf.train.latest_checkpoint(model_dir)             # reprendre l'entrainement à partir du dernier checkpoint trouvé

if latest:
    model.load_weights(latest)   # si un checkpoint est trouvé, charger les poids enregistrés à ce chekpoint

# définition du checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, save_best_only=True)
# définition de l'early stopping pour arrêter le modèle si la loss "stagne" - "atteint une limite" - "minimum absolu" 
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', 
                                            baseline=None, restore_best_weights=True)

# définition des calllbacks (checkpoint  + early stopping)
callbacks = [cp_callback, es_callback]

In [ ]:
epochs = 100        # nombre d'epochs lors de l'entrainement
batch_size = 128    # batch_size volontairement haut afin d'éviter l'over-fitting

### Entraînement du modèle

In [ ]:
history = model.fit(x=X_train, y=y_train, 
          validation_data=(X_validation, y_validation),
          epochs=epochs, batch_size=batch_size, callbacks=callbacks, use_multiprocessing=False, verbose=1)

### Résultats de l'entraînement

In [ ]:
# affichag de la courbe de précision en fonction des epochs
fig, ax = plt.subplots(2,1, figsize=(15, 10))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

# affichag de la courbe de loss en fonction des epochs
ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

### Evaluation du modèle

In [ ]:
# affichage de la loss et de l'accuracy sur l'ensemble de test pour définir la performance finale de notre modèle
model.evaluate(X_test, y_test, batch_size=128)

In [ ]:
model.save('model.h5') # enregistrements des poids de notre modèle pour le réutiliser dans l'étape de prédiction

# Partie 5

## Prédiction sur des données géospatiales

### Chargement de la donnée à prédire

In [ ]:
image_path = 'image_2019-10-01_2020-10-01.tif'          # donnée géospatiale
predicted_path = 'predicted_2019-10-01_2020-10-01.tif'  # nom du fichier de la donnée prédite
data_source = gdal.Open(image_path)                     # chargement de la donnée
image = data_source.ReadAsArray()                       # lire la donnée en array
image.shape

### Preprocesssing de la donnée à prédire

In [ ]:
flat_image = image.reshape(image.shape[0],
                           image.shape[1] * image.shape[2])                             # redimensionner l'array pour le lire dans le modèle

flat_image = flat_image.transpose()                                                     # inverser l'array

flat_image = flat_image.reshape((flat_image.shape[0],                                   # redimensionnement
                                 int(flat_image.shape[1] / n_features), n_features))

# appliquer le même preprocessing que pour l'entrainement
flat_image = np.array(flat_image).astype(float)                                         
print(flat_image.shape)

padded_image = tf.keras.preprocessing.sequence.pad_sequences(flat_image, maxlen=sequence_size, dtype='float32')
padded_image.shape

# normaliser les valeurs de l'array
rescaled_image = padded_image / 10000.0

### Lancer l'inférence sur la donnée à l'aide du modèle

In [ ]:
flat_predicted = model.predict(rescaled_image, batch_size=1024)
flat_predicted.shape

### Préparer les résultats de la prédiction pour l'affichage du résultat

In [ ]:
# préparation des labels
flat_labels = np.argmax(flat_predicted, axis=2)
print(flat_predicted.shape, '-->', flat_labels.shape)

print(flat_labels.shape, flat_image.shape[1])

valid_flat_labels = flat_labels[:,-flat_image.shape[1]:]
print(valid_flat_labels.shape)

# préparation de l'image
predicted_image = valid_flat_labels.reshape((image.shape[1], image.shape[2], valid_flat_labels.shape[-1]))
print(predicted_image.shape)

predicted_image = predicted_image[:, :, predicted_image.shape[-1] - image.shape[0]:]
predicted_image.shape

### Enregistrement des résultats au format TIF (donnée géospatiale)

In [ ]:
driver = data_source.GetDriver()
output_dataset = driver.Create(predicted_path,
                               predicted_image.shape[1],
                               predicted_image.shape[0],
                               predicted_image.shape[-1],
                               gdal.GDT_Byte,
                               ['COMPRESS=DEFLATE'])
output_dataset.SetGeoTransform(data_source.GetGeoTransform())
output_dataset.SetProjection(data_source.GetProjection())

for band_id in range(predicted_image.shape[-1]):
    band_data = predicted_image[:, : , band_id]        
    output_dataset.GetRasterBand(band_id + 1).WriteArray(band_data, 0, 0)
output_dataset.FlushCache()
del output_dataset
print("Completed!")

# Partie 6

## Chargement de la prédiction pour l'affichage d'un résultat

### Chargement de la donnée géospatiale (TIF) prédite

In [ ]:
data_source = gdal.Open(predicted_path)
image = data_source.ReadAsArray()
image.shape

### Construction de la vidéo pour compiler les images TIF de données géospatiales

In [ ]:
# fonction pour charger un fichire et jouer la vidéo dans un format html

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

### Supperposer la prédiction avec la légende des labels sur la vidéo

In [ ]:
# préparer la structure du plot
fig, ax = plt.subplots(figsize=(15, 10))

ims = []

# récupérer les couleurs des classes (labels)
cmap = colors.ListedColormap(classe_colors) 

# pour chaque morceaux de l'image (TIF) prédite
    # afficher l'image prédite (image + supperposition des labels et des couleurs)
    # ajoutter la superposition à la liste imss
for band in image:
    im = ax.imshow(band, vmin=0, vmax=len(classe_colors)-1, cmap=cmap, animated=True)
    ims.append([im])

# créer une animation dont les frames sont les images de la liste ims
ani = animation.ArtistAnimation(fig, ims, interval=1000, blit=True, repeat_delay=1000)

# ajouter les labels dans la légente avec la coloration par classe
patches = list(map(lambda item: mpatches.Patch(color=item[1].get('color'), label=item[0]), classe_info.items() ))
plt.legend(handles=patches, loc='center left', bbox_to_anchor=(1, 0.5))
    
output = 'predicted.mp4' # nom du fichire d'enregistrement
ani.save(output)         # enregistrer le résultat au format mp4

play(output)             # jouer le résultat final dans un cadre html (pour le notebook)